Hatırlatıcı Not:  
*Standardizasyonun Faydalı Olduğu Durumlar:*  
* Mesafeye dayalı algoritmalar (KNN, SVM, Lojistik Regresyon, PCA)
* Gradient bazlı modeller (Yapay Sinir Ağları)
* Farklı ölçeklerdeki değişkenler içeren veri setleri

*Zararlı Olabileceği Durumlar:*
* Ağaç tabanlı modeller (Decision Tree, Random Forest, XGBoost) → Genelde ölçeklemeye ihtiyaç duymaz
* Anlamlı kategorik veya sıralı veriler (örn: eğitim seviyesi 1-5 arası) → Ölçekleme anlamı bozabilir

# Bagging

Temeli bootstrap yöntemi ile oluşturulan birden fazla karar ağacının ürettiği tahminlerin bir araya getirilerek değerlendirilmesine dayanır.

Topluluk öğrenme, birden fazla modelin birlikte çalışarak daha iyi tahmin yapmasını sağlar. XGBoost, Random Forest, Bagged Trees gibi algoritmalar birden çok karar ağacı kullanarak daha güçlü ve kararlı modeller oluşturur.

![image.png](image14.png)

Bagging (Bootstrap Aggregating), veri setinden rastgele örnekler alarak (bootstrap) birden fazla karar ağacı oluşturur ve bu ağaçların tahminlerini birleştirerek daha güçlü ve kararlı bir model elde eder.

Oluşturulan ağaçların diğer ağaçlar ile bir ilişkisi yoktur. Örneklerin her bir ağaçta kullanılma şansı eşittir.

## Model

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split

hit = pd.read_csv('Hitters.csv')
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df['Salary']
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.25,
                                                   random_state=42)

In [2]:
from sklearn.ensemble import BaggingRegressor
bag_model = BaggingRegressor(bootstrap_features=True)

In [3]:
bag_model.fit(X_train, y_train)

BaggingRegressor(bootstrap_features=True)

In [4]:
bag_model.get_params()

{'bootstrap': True,
 'bootstrap_features': True,
 'estimator': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [5]:
bag_model.n_estimators

10

In [6]:
bag_model.estimators_ # bunlarin hepsi birbirinden farkli agaclar. her biri bootstrap yöntemi ile rastgelelik saglanir

[DecisionTreeRegressor(random_state=1704760709),
 DecisionTreeRegressor(random_state=1612050791),
 DecisionTreeRegressor(random_state=413152526),
 DecisionTreeRegressor(random_state=1283662366),
 DecisionTreeRegressor(random_state=189382959),
 DecisionTreeRegressor(random_state=1376927947),
 DecisionTreeRegressor(random_state=602419123),
 DecisionTreeRegressor(random_state=162716282),
 DecisionTreeRegressor(random_state=893185637),
 DecisionTreeRegressor(random_state=1099381479)]

In [7]:
bag_model.estimators_samples_ # her bir agacin içindeki ornekler. Bu sayılar veri setindeki örneklerin indeksleri

[array([170,  69,  19, 171, 186,  28,   2, 174, 134, 186, 156, 159, 117,
        188, 141, 101, 178, 115, 177,  12,   3,  91, 105,  49, 155,  14,
         46, 179, 158, 144, 170,  89, 162,  93,  55, 126,  95,  48, 102,
         38, 166,  85, 173, 119,   3, 112,  68,  72,  70, 141, 136,   2,
        113,  37, 161, 173,  44,  95,  26, 125, 163,  66, 120,  80,  18,
         86,  22, 187, 138,  24,  43,  51,  86, 146, 165, 171,  44,  62,
         99,  49,   8,  67,  67, 133, 163, 182, 131, 119, 192,  47, 145,
         81, 127,  54,  47,  21,  40, 144,  68,  16, 173, 162,  92, 127,
        187,  91,  66, 133,  71, 143, 182,  32,  76, 124,  90,  95, 139,
         43,  47, 133, 163, 152, 173, 134,  68,  29, 166,  52, 125,  98,
        145,  12, 155, 133,  96, 136,  17,  95,  54, 143, 134,  10, 119,
         81,  63,  67,  26,  93,  62, 101,  50,  39, 187,  88, 120, 116,
         42,  50, 132,  94, 120, 142,  80,  78, 123, 191, 154, 151, 156,
         94, 143,   9,  87,  67, 107,  52,  76, 182

In [8]:
bag_model.estimators_features_

[array([ 5,  1, 14, 13, 17,  5, 17, 13,  8,  9,  1, 17,  9, 14,  2,  4, 14,
        15,  8], dtype=int32),
 array([ 7,  7, 13,  1,  2,  1, 14,  1,  6, 16, 12, 11, 17,  7,  3, 14, 18,
         7,  7], dtype=int32),
 array([14, 16, 15, 16,  7, 10,  7,  0, 18,  8, 10,  2, 18, 13,  5,  1,  4,
        15, 12], dtype=int32),
 array([ 3, 12, 14,  4,  9,  1, 18, 18, 15, 16, 10, 18, 14,  4, 17,  1, 13,
        16,  2], dtype=int32),
 array([15,  6,  0, 14, 15, 16,  7, 13,  4,  8, 17,  4,  1,  9, 17,  7,  0,
        12,  5], dtype=int32),
 array([11, 13, 11, 12,  9,  2,  0,  2,  0, 10, 17,  4, 15, 13,  1, 13, 11,
         8,  8], dtype=int32),
 array([ 8,  9, 12, 11, 16,  7, 13, 11, 10,  8, 14, 15, 10,  5,  6,  9,  0,
        17,  0], dtype=int32),
 array([11, 11,  2, 10,  3,  4, 12,  6,  2, 13,  2, 11, 17,  0,  3, 16,  5,
        12, 17], dtype=int32),
 array([ 5, 18, 10,  4,  2, 15,  3,  2, 18, 17, 14,  7, 14, 16,  3,  2, 13,
        12,  5], dtype=int32),
 array([ 7,  5, 13,  4,  5,  2, 14, 1

- **`bag_model.estimators_`** → Bagging içinde eğitilen **farklı karar ağaçlarını** gösterir.  
- **`bag_model.estimators_samples_`** → **Her ağacın eğitildiği veri seti örneklerinin indekslerini** gösterir (bootstrap ile rastgele seçilmiş).  
- **`bag_model.estimators_features_`** → **Her ağacın kullandığı özelliklerin (feature)(sütun) indekslerini** gösterir (bazı özellikler rastgele seçilmiş).

In [9]:
bag_model.estimators_[1]

DecisionTreeRegressor(random_state=1612050791)

## Tahmin

In [10]:
from sklearn.metrics import mean_squared_error
import numpy as np 
y_pred = bag_model.predict(X_test)
np.sqrt(mean_squared_error(y_pred, y_test))

np.float64(351.89772330385784)

In [11]:
iki_y_pred = bag_model.estimators_[1].fit(X_train, y_train).predict(X_test) # Burada, rastgele olusan bir agaci egitiyoruz.
np.sqrt(mean_squared_error(iki_y_pred, y_test))

np.float64(452.1594590236844)

In [12]:
# Bu agac biraz beceriksiz cikti. Random Forest'da becerikli veya beceriksiz agaclara agirlik verebiliyoruz.

In [13]:
iki_y_pred = bag_model.estimators_[3].fit(X_train, y_train).predict(X_test) # Burada, rastgele olusan bir agaci egitiyoruz.
np.sqrt(mean_squared_error(iki_y_pred, y_test))

np.float64(451.5852082675824)

## Model Tuning

Bag etmek = Bootstrapleri birleştirmek,  
Bundan dolayı model tuning yaparken bootstraplerin sayısını tune edeceğiz.

In [18]:
from sklearn.model_selection import GridSearchCV
bag_model = BaggingRegressor(bootstrap_features=True)
bag_model.fit(X_train, y_train)

BaggingRegressor(bootstrap_features=True)

In [27]:
bag_params = {'n_estimators': range(2, 50)}

In [28]:
bag_cv_model = GridSearchCV(bag_model, bag_params, cv=10)

In [29]:
bag_cv_model.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=BaggingRegressor(bootstrap_features=True),
             param_grid={'n_estimators': range(2, 50)})

In [30]:
bag_cv_model.best_params_

{'n_estimators': 4}

In [43]:
bag_tuned = BaggingRegressor(n_estimators=4, random_state=514)

In [44]:
bag_tuned.fit(X_train, y_train)

BaggingRegressor(n_estimators=4, random_state=514)

In [45]:
y_pred = bag_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

np.float64(328.2904357538759)